In [2]:
# Install the required libraries
!pip install ipywidgets plotly pillow --quiet
from google.colab import output
output.enable_custom_widget_manager()

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display, clear_output
from PIL import Image
import io

# Constants
g0 = 9.80665  # m/s^2

# Fuel Type Definition
class FuelType:
    def __init__(self, name, isp_vac, isp_sea, density):
        self.name = name
        self.isp_vac = isp_vac  # seconds in vacuum
        self.isp_sea = isp_sea  # seconds at sea level
        self.density = density  # kg/m^3

# Predefined fuel types
RP1 = FuelType("RP-1", isp_vac=311, isp_sea=275, density=810)
LH2 = FuelType("LH2", isp_vac=450, isp_sea=370, density=71)
Methalox = FuelType("Methalox", isp_vac=370, isp_sea=320, density=422)

FUEL_TYPES = {
    "RP-1": RP1,
    "LH2": LH2,
    "Methalox": Methalox
}

# Rocket Stage Definition
class RocketStage:
    def __init__(self, name, dry_mass, fuel_mass, fuel_type, burn_time, thrust_curve=None):
        self.name = name
        self.dry_mass = dry_mass
        self.fuel_mass = fuel_mass
        self.initial_fuel_mass = fuel_mass
        self.fuel_type = fuel_type if isinstance(fuel_type, FuelType) else FUEL_TYPES[fuel_type]
        self.burn_time = burn_time
        self.thrust_curve = thrust_curve

    def isp(self, altitude):
        return np.interp(altitude, [0, 10000], [self.fuel_type.isp_sea, self.fuel_type.isp_vac])

    def average_thrust(self, altitude):
        return self.isp(altitude) * g0 * (self.initial_fuel_mass / self.burn_time)

    def thrust(self, t, altitude):
        if self.thrust_curve:
            return self.thrust_curve(t)
        return self.average_thrust(altitude)

    def mass_flow_rate(self):
        return self.initial_fuel_mass / self.burn_time

    def delta_v(self):
        m0 = self.dry_mass + self.initial_fuel_mass
        mf = self.dry_mass
        return self.fuel_type.isp_vac * g0 * np.log(m0 / mf)

    @property
    def total_mass(self):
        return self.dry_mass + self.fuel_mass

# Rocket Definition
class Rocket:
    def __init__(self, stages, payload_mass):
        self.stages = stages
        self.payload_mass = payload_mass

    def total_mass(self):
        return sum(stage.total_mass for stage in self.stages) + self.payload_mass

    def total_delta_v(self):
        m = self.total_mass()
        dv_total = 0
        for stage in self.stages:
            m0 = m
            mf = m - stage.initial_fuel_mass
            dv_total += stage.fuel_type.isp_vac * g0 * np.log(m0 / mf)
            m = mf - stage.dry_mass
        return dv_total

    def simulate(self, dt=0.1, angle_deg=90):
        t = 0
        logs = []
        velocity = 0
        altitude = 0
        mass = self.total_mass()
        remaining_stages = self.stages[:]
        angle_rad = np.radians(angle_deg)

        for stage in remaining_stages:
            m_stage = stage.total_mass
            dm_dt = stage.mass_flow_rate()
            t_stage = 0

            while t_stage < stage.burn_time:
                thrust = stage.thrust(t_stage, altitude)
                acc = thrust / mass - g0 * np.cos(angle_rad)
                velocity += acc * dt
                altitude += velocity * dt * np.sin(angle_rad)
                fuel_burn = dm_dt * dt
                stage.fuel_mass -= fuel_burn
                stage.fuel_mass = max(0, stage.fuel_mass)
                mass -= fuel_burn
                logs.append({
                    'time': t,
                    'altitude': altitude,
                    'velocity': velocity,
                    'mass': mass,
                    'thrust': thrust,
                    'acceleration': acc,
                    'stage': stage.name
                })

                t += dt
                t_stage += dt

            mass -= stage.dry_mass

        return pd.DataFrame(logs)

# Interactive GUI
def run_interactive_gui():
    s1_dry = widgets.FloatText(value=5000, description='S1 Dry Mass (kg)')
    s1_fuel = widgets.FloatText(value=15000, description='S1 Fuel Mass (kg)')
    s1_fuel_type = widgets.Dropdown(options=list(FUEL_TYPES.keys()), value='RP-1', description='S1 Fuel')
    s1_burn = widgets.FloatText(value=60, description='S1 Burn Time (s)')

    s2_dry = widgets.FloatText(value=2000, description='S2 Dry Mass (kg)')
    s2_fuel = widgets.FloatText(value=5000, description='S2 Fuel Mass (kg)')
    s2_fuel_type = widgets.Dropdown(options=list(FUEL_TYPES.keys()), value='LH2', description='S2 Fuel')
    s2_burn = widgets.FloatText(value=40, description='S2 Burn Time (s)')

    payload = widgets.FloatText(value=1000, description='Payload Mass (kg)')

    run_btn = widgets.Button(description='Run Simulation', button_style='success')
    out = widgets.Output()

    def on_click_run(_):
        s1 = RocketStage("Stage 1", dry_mass=s1_dry.value, fuel_mass=s1_fuel.value, fuel_type=s1_fuel_type.value, burn_time=s1_burn.value)
        s2 = RocketStage("Stage 2", dry_mass=s2_dry.value, fuel_mass=s2_fuel.value, fuel_type=s2_fuel_type.value, burn_time=s2_burn.value)
        rkt = Rocket([s1, s2], payload_mass=payload.value)
        result = rkt.simulate()

        with out:
            clear_output()
            fig, axs = plt.subplots(4, 1, figsize=(12, 14))
            axs[0].plot(result['time'], result['altitude'])
            axs[0].set_ylabel('Altitude (m)')
            axs[0].set_title('Altitude vs Time')

            axs[1].plot(result['time'], result['velocity'])
            axs[1].set_ylabel('Velocity (m/s)')
            axs[1].set_title('Velocity vs Time')

            axs[2].plot(result['time'], result['thrust'])
            axs[2].set_ylabel('Thrust (N)')
            axs[2].set_title('Thrust vs Time')

            axs[3].plot(result['time'], result['acceleration'])
            axs[3].set_ylabel('Acceleration (m/s²)')
            axs[3].set_xlabel('Time (s)')
            axs[3].set_title('Acceleration vs Time')

            plt.tight_layout()
            plt.show()

            # 3D Plot: Try interactive, fallback to static if needed
            fig_3d = go.Figure(data=[go.Scatter3d(
                x=result['time'],
                y=result['altitude'],
                z=result['velocity'],
                mode='lines',
                line=dict(color=result['thrust'], colorscale='Viridis', width=4)
            )])
            fig_3d.update_layout(
                scene=dict(
                    xaxis_title='Time (s)',
                    yaxis_title='Altitude (m)',
                    zaxis_title='Velocity (m/s)'
                ),
                title="3D Rocket Flight Simulation",
                margin=dict(l=0, r=0, b=0, t=40)
            )
            try:
                fig_3d.show(renderer="colab")
            except Exception as e:
                print("Interactive 3D plot failed, showing static image instead.")
                buf = io.BytesIO()
                fig_3d.write_image(buf, format='png')
                buf.seek(0)
                img = Image.open(buf)
                plt.figure(figsize=(8,6))
                plt.imshow(img)
                plt.axis('off')
                plt.title("3D Rocket Flight Simulation (Static)")
                plt.show()

            max_alt = result['altitude'].max()
            max_vel = result['velocity'].max()
            max_acc = result['acceleration'].max()
            print(f"Total Δv available: {rkt.total_delta_v():.2f} m/s")
            print(f"Max altitude: {max_alt:.2f} m")
            print(f"Max velocity: {max_vel:.2f} m/s")
            print(f"Max acceleration: {max_acc:.2f} m/s²")
            print(f"Final mass: {result['mass'].iloc[-1]:.2f} kg")

    run_btn.on_click(on_click_run)

    ui = widgets.VBox([
        widgets.HTML('<h3>Rocket Configuration</h3>'),
        s1_dry, s1_fuel, s1_fuel_type, s1_burn,
        s2_dry, s2_fuel, s2_fuel_type, s2_burn,
        payload,
        run_btn,
        out
    ])
    display(ui)

run_interactive_gui()



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 19.0 MB/s eta 0:00:00
